In [32]:
import re
import dateparser

def homework1 ():
    file = open("Apache_2k.log")
    pattern = ""
    count = 0
    print("Date                       Client IP")
    for row in file:
        
        ips = re.findall(r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b", row)
        if ips:
            row = row.split("]")
            count += 1
            row = row[0][1:len(row[0])]
            date = dateparser.parse(row).strftime("%m/%d/%Y %I:%M:%S %p")
            print(f"{date}     {ips[0]}")
    print("%s %d" % ("Total amount of forbidden requests:", count))

In [33]:
homework1()

Date                       Client IP
12/04/2005 05:15:09 AM     222.166.160.184
12/04/2005 07:45:45 AM     63.13.186.196
12/04/2005 08:54:17 AM     147.31.138.75
12/04/2005 09:35:12 AM     207.203.80.15
12/04/2005 10:53:30 AM     218.76.139.20
12/04/2005 11:11:07 AM     24.147.151.74
12/04/2005 11:33:18 AM     211.141.93.88
12/04/2005 11:42:43 AM     216.127.124.16
12/04/2005 12:33:13 PM     208.51.151.210
12/04/2005 01:32:32 PM     65.68.235.27
12/04/2005 02:29:00 PM     4.245.93.87
12/04/2005 03:18:36 PM     67.154.58.130
12/04/2005 03:59:01 PM     24.83.37.136
12/04/2005 04:24:05 PM     58.225.62.140
12/04/2005 05:34:57 PM     61.138.216.82
12/04/2005 05:53:43 PM     218.39.132.175
12/04/2005 06:24:22 PM     125.30.38.52
12/04/2005 07:36:05 PM     61.37.222.240
12/05/2005 01:04:31 AM     218.62.18.218
12/05/2005 01:30:32 AM     211.62.201.48
12/05/2005 03:23:24 AM     218.207.61.7
12/05/2005 03:44:50 AM     168.20.198.21
12/05/2005 06:36:59 AM     221.232.178.24
12/05/2005 09:09:48 

C:\Users\joaog\AppData\Local\Programs\Python\Python310\lib\site-packages\dateparser\date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


In [72]:
import csv

def homework2():
    class Country:
        def __init__(self, name, count):
            self.name = name
            self.count = count
    file = open("IMDb_movies.csv", encoding='utf8')
    file = csv.reader(file)
    countries_all = []
    countries_once = []
    countries_end = []
    for row in file:
        if len(row[7].split(",")) >= 2:
            for country in row[7].split(", "):
                countries_all.append(country)
                if country not in countries_once:
                    countries_once.append(country)
        else:
            countries_all.append(row[7])
            if row[7] not in countries_once:
                countries_once.append(row[7])
    for country in countries_once:
        amount = countries_all.count(country)
        countries_end.append(Country(country, amount))
    countries_end = sorted(countries_end, key=lambda x:int(x.count), reverse = True)
    for i in range(10):
        name = countries_end[i].name
        count = countries_end[i].count
        number_bars = int(count/1000)
        spaces = "\u2588"*number_bars
        print(f"%-10s \33[40m%s\33[0m (%4s k)" % (name, spaces, round(count/1000, 2)))

In [73]:
homework2()

USA        ██████████████████████████████████ (34.33 k)
France     ████████ (8.31 k)
UK         ███████ (7.49 k)
India      ██████ (6.37 k)
Italy      █████ (5.06 k)
Germany    ███ (3.72 k)
Japan      ███ ( 3.7 k)
Canada     ███ (3.62 k)
Spain      ██ (2.73 k)
Hong Kong  █ (1.88 k)


In [36]:
import bs4
import urllib
import requests
from re import split

def read_words(url, skip_until):
    stop_words = [
    "a", "about", "above", "across", "after", "afterwards", "again", "against",
    "all", "almost", "alone", "along", "already", "also", "although", "always",
    "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
    "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are",
    "around", "as", "at", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
    "below", "beside", "besides", "between", "beyond", "bill", "both",
    "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con",
    "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
    "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else",
    "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
    "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill",
    "find", "fire", "first", "five", "for", "former", "formerly", "forty",
    "found", "four", "from", "front", "full", "further", "get", "give", "go",
    "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter",
    "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his",
    "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed",
    "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter",
    "latterly", "least", "less", "ltd", "made", "many", "may", "me",
    "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly",
    "move", "much", "must", "my", "myself", "name", "namely", "neither",
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone",
    "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on",
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
    "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
    "please", "put", "rather", "re", "same", "see", "seem", "seemed",
    "seeming", "seems", "serious", "several", "she", "should", "show", "side",
    "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone",
    "something", "sometime", "sometimes", "somewhere", "still", "such",
    "system", "take", "ten", "than", "that", "the", "their", "them",
    "themselves", "then", "thence", "there", "thereafter", "thereby",
    "therefore", "therein", "thereupon", "these", "they", "thick", "thin",
    "third", "this", "those", "though", "three", "through", "throughout",
    "thru", "thus", "to", "together", "too", "top", "toward", "towards",
    "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us",
    "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
    "whence", "whenever", "where", "whereafter", "whereas", "whereby",
    "wherein", "whereupon", "wherever", "whether", "which", "while", "whither",
    "who", "whoever", "whole", "whom", "whose", "why", "will", "with",
    "within", "without", "would", "yet", "you", "your", "yours", "yourself",
    "yourselves"
    ]
    input_file = urllib.request.urlopen(url)
    word_set = set()
    skip = True
    for line in input_file :
        line = line.decode('utf-8').strip()  
        if not skip :
            if line.find(skip_until) >= 0:
                break
            parts = split("[^a-zA-Z]+", line) # notice this unique split function from the re module
            for word in parts :
                if len(word) > 0 :
                    word_set.add(word.lower())
        elif line.find(skip_until) >= 0 : 
            skip = False
    return word_set

def homework3():
    url = "https://www.gutenberg.org/files/11/old/alice30.txt"
    response = urllib.request.urlopen(url)

    my_dictionary = dict()
    chapters = []
    titles = dict()
    lines = response.readlines()
    for i in range(len(lines)):
        row = lines[i].decode('utf-8').strip()
        if "CHAPTER" in row:
            chpt = row
            title = lines[i+2].decode('utf-8').lstrip().replace("\r\n", "")
            chapters.append(chpt)
            titles[chpt] = title
            my_dictionary[chpt] = ""
    chapters.reverse()
    for chap in chapters:
        my_dictionary[chap] = read_words(url, chap)
    print("%-15s %-40s %-s" % ("CHAPTER", "CHAPTER TITLE", "UNIQUE WORD COUNT"))
    for chap in my_dictionary:
        print("%-15s %-40s %-s" % (chap, titles[chap], len(my_dictionary[chap])))

In [37]:
homework3()

CHAPTER         CHAPTER TITLE                            UNIQUE WORD COUNT
CHAPTER I       Down the Rabbit-Hole                     605
CHAPTER II      The Pool of Tears                        601
CHAPTER III     A Caucus-Race and a Long Tale            2342
CHAPTER IV      The Rabbit Sends in a Little Bill        2218
CHAPTER V       Advice from a Caterpillar                601
CHAPTER VI      Pig and Pepper                           664
CHAPTER VII     A Mad Tea-Party                          586
CHAPTER VIII    The Queen's Croquet-Ground               1582
CHAPTER IX      The Mock Turtle's Story                  1372
CHAPTER X       The Lobster Quadrille                    530
CHAPTER XI      Who Stole the Tarts?                     522
CHAPTER XII     Alice's Evidence                         633


In [68]:
import csv

def extra_credit_a():
    class Country:
        def __init__(self, name, count):
            self.name = name
            self.count = count
    file = open("IMDb_movies.csv", encoding='utf8')
    file = csv.reader(file)
    countries_all = []
    countries_once = []
    countries_end = []
    for row in file:
        if len(row[7].split(",")) >= 2:
            for country in row[7].split(", "):
                countries_all.append(country)
                if country not in countries_once:
                    countries_once.append(country)
        else:
            countries_all.append(row[7])
            if row[7] not in countries_once:
                countries_once.append(row[7])
    for country in countries_once:
        amount = countries_all.count(country)
        countries_end.append(Country(country, amount))
    countries_end = sorted(countries_end, key=lambda x:int(x.count), reverse = True)
    colors = ["\33[31m", "\33[32m", "\33[33m", "\33[34m", "\33[35m",
             "\33[36m", "\33[90m", "\33[91m", "\33[92m", "\33[93m"]
    for i in range(10):
        name = countries_end[i].name
        count = countries_end[i].count
        number_bars = int(count/1000)
        spaces = "\u2588"*number_bars
        print(f"%-10s %s%s\33[0m (%4s k)" % (name, colors[i], spaces, round(count/1000, 2)))

In [69]:
extra_credit_a()

USA        ██████████████████████████████████ (34.33 k)
France     ████████ (8.31 k)
UK         ███████ (7.49 k)
India      ██████ (6.37 k)
Italy      █████ (5.06 k)
Germany    ███ (3.72 k)
Japan      ███ ( 3.7 k)
Canada     ███ (3.62 k)
Spain      ██ (2.73 k)
Hong Kong  █ (1.88 k)


In [74]:
def extra_credit_b(data, replacements, color):
    # iterating through the all characters to be replace
    # and the replacing list
    # then replacing if one contains the other
    for character in data:
        for i in range(len(character)):
            for replace in replacements:
                if character[i] == replace[0]:
                    character[i] = replace[1]
            print(color + character[i], end="")
        print("")
        

data =[
['!','F','L','O','R','/','D','A','\'','_'],
['!','!','!','_','T','*','M','!','!','!','!'],
['U','N','/','V','*','R','_','/','T','Y']
]
replacements = [('*', 'E'), ('/', 'I'), ('!', ' '), ('_', 'S')]

extra_credit_b(data, replacements, "\33[41m")

art1 =[
['|','|','|','|','|','|','/','!','·','#','_'],
['|','|','|','|','|','/','_','#','#','#','_','!','U','·'],
['|','_','#','·',' ́','|','|','_','|','|','|','!','·','#','_','#','·',' ́','A'],
['U','|','©','|','A','U',' ́',';','|','|','|','|','|','|','_','|','|','{'],
['|','!','·','#','_','|','!','·','|','|','_','#','·',' ́','\\','!','·','_','A'],
['|','|','|','|','|','!','\\','\\',' ́',' ́','\\','_','#','·',' ́']
]
replacements = [('|', ' '), ('!', '`'), ('_', ' ̧'), ('#', '.'), ('A',')'), ('U', ':')]

extra_credit_b(art1, replacements, "\33[46m")

art2 = [
[':',':',':',':','.','_',':',':',':',':','.','_'],
[':',':',':','.',':',':','_',':',':','.',':',':','_'],
[':',':','.',':','.','_',':','_','.',':','.','_',':','_'],
[':','.',':','.',':',':','_',':','_','.',':',':','_',':','_'],
['.',':','.',':',':',':','.','_',':','_',':',':',':','_',':','_'],
['_',':','_',':',':','.',':','.','_',':','_',':',':','.',':','.'],
[':','_',':','_','.',':','.',':',':','_',':','_','.',':','.'],
[':',':','_','.',':','.',':',':',':',':','_','.',':','.'],
[':',':','.',':','.','_',':',':',':',':','.',':','.','_'],
[':','.',':','.','_',':','_',':',':','.',':','.','_',':','_'],
['.',':','.',':',':','_',':','_','.',':','.',':',':','_',':','_'],
['_',':','_',':',':',':','_',':','_','.',':',':',':','.',':','.'],
[':','_',':','_',':',':','.','_',':','_',':',':','.',':','.'],
[':',':','_',':','_','.',':','.','_',':','_','.',':','.'],
[':',':',':','_',':',':','.',':',':','_',':',':','.'],
[':',':',':',':','_','.',':',':',':',':','_','.']
]
replacements = [('.', '/'), ('_','\\'), (':', ' ')]

extra_credit_b(art2, replacements, "\33[102m")

art3 = [
['T','T','T','T','!','-','-','!','T','T','T','T','T','T','T','T','T','T','T','T','T','T','!','-','-','!'],
['T','T','T',':','T','(','\\','T','"','!','T','|','!','!','!','!','!','!','|','T','!','"','T','/',')','T',':'],
['T','T','T','T','\'','!','T','T','T','T','`','T','T','T','T','T','T','T','T','`','T','T','T','T','!','\''],
['T','T','T','T','T','/','\'','T','T','T','|','T','T','T','T','T','T','T','T','|','T','T','T','`','\\'],
['T','T','T','T','/','T','T','T','T','T','y','}','T','T','T','T','T','T','{','y','T','T','T','T','T','\\'],
['T','T','T','|','T','T','T','T','T','T','T','/','T','T','T','T','T','T','\\','T','T','T','T','T','T','T','|'],
['T','T','T','|','T','T','T','T','T','/','\'','T','T','T','T','T','T','T','T','`','\\','T','T','T','T','T','|'],
['T','T','T','T','\\','T','T','T','|','T','!','T','T','!','=','=','!','T','T','!','T','|','T','T','T','/'],
['T','T','T','T','T','\'','!','|','T','\\','!','\'','T','\\','|','|','/','T','\'','!','/','T','|','!','\''],
['T','T','T','T','T','/','T','T','`','`','\'','!','|','-','\'','\'','-','|','!','\'','`','`','T','T','\\'],
]
replacements = [('|', '_'), ('T', ' '), ('!', '.'), ('y', '0')]

extra_credit_b(art3, replacements, "\33[45m")

 FLORIDA'S
   STEM    
UNIVERSITY
      /`·. ̧
     / ̧... ̧`:·
  ̧.· ́   ̧   `·. ̧.· ́)
: © ): ́;       ̧  {
 `·. ̧ `·   ̧.· ́\`· ̧)
     `\\ ́ ́\ ̧.· ́
    /\    /\
   /  \  /  \
  / /\ \/ /\ \
 / /  \ \/  \ \
/ /   /\ \   \ \
\ \  / /\ \  / /
 \ \/ /  \ \/ /
  \/ /    \/ /
  / /\    / /\
 / /\ \  / /\ \
/ /  \ \/ /  \ \
\ \   \ \/   / /
 \ \  /\ \  / /
  \ \/ /\ \/ /
   \  /  \  /
    \/    \/
    .--.              .--.
   : (\ ". _......_ ." /) :
    '.    `        `    .'
     /'   _        _   `\
    /     0}      {0     \
   _       /      \       _
   _     /'        `\     _
    \   _ .  .==.  . _   /
     '._ \.' \__/ './ _.'
     /  ``'._-''-_.'``  \
